## Determine if W Converges


In [ ]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from spikey.core import *

from spikey.snn import *
from spikey.RL import *

In [ ]:
def run(network_temp, game_temp):
    dw = np.ma.array(np.zeros(network_temp.config['matrix'].shape), mask=network_temp.config['matrix'].mask)
    prev_state = None

    params = deepcopy(network_temp.config)

    game = game_temp(**params)
    network = network_temp(game=game, **params)

    print(network.synapses.w.matrix)

    ##
    for e in range(experiment_params['n_episodes']):
        total_reward = 0

        network.reset()
        state = game.reset()

        for s in range(experiment_params['len_episode']):
            if (experiment_params['n_episodes'] * experiment_params['len_episode']) - (e * experiment_params['len_episode']) - s <= experiment_params['eval_steps']:
                curr_state = network.synapses.w.matrix

                if prev_state is not None:
                    dw += curr_state - prev_state

                prev_state = np.copy(curr_state)

            action = network.tick(state)
            
            ##
            relevant_spikes = np.abs(network.spike_log[-network._processing_time:, -network._n_outputs:])

            outrate = np.mean(relevant_spikes)
            n_spikes = np.sum(relevant_spikes)

            ##
            state, _, done, __ = game.step(action)

            reward = network.reward(state, action)

            total_reward += reward

            if done:
                break

    ##
    print(network.synapses.w.matrix)

    dw = np.ravel(dw[~dw.mask])

    if not np.sum(dw):
        print("No change.")
    else:
        sns.distplot(dw)
        plt.show()

    print(f"Mean: {np.mean(dw)}, STD: {np.std(dw)}")

    return network, game

In [ ]:
## No training - Topology - should say converged
experiment_params = {
    'n_episodes': 1,
    'len_episode': 50,
    'eval_steps': 50, 
}

N_INPUTS = 60
N_NEURONS = 61
N_OUTPUTS = 1


w = np.vstack((  # Fully connected, generated randomly over interval
    np.hstack((
        np.random.uniform(0, .2, (N_INPUTS, N_NEURONS - N_OUTPUTS)),
        np.zeros((N_INPUTS, N_OUTPUTS)))),
    np.hstack((
        np.zeros((N_NEURONS - N_OUTPUTS, N_NEURONS - N_OUTPUTS)),
        np.random.uniform(0, .2, (N_NEURONS - N_OUTPUTS, N_OUTPUTS)))),
    np.zeros((N_OUTPUTS, N_NEURONS)),
))
w = np.ma.array(np.float16(w), mask=(w == 0), fill_value=0)

##
class network_templatee(SNN):
    config = {
        'matrix': w,                  # v/
        'n_neurons': N_NEURONS,       # v/
        'n_outputs': N_OUTPUTS,
        'input_pct_inhibitory': .5,   # v/
        'neuron_pct_inhibitory': 0,          # v/
        'processing_time': 500,       # v/ 500ms NOTENOTENOTENOTENOTENOTENOTENOTENOTENOTENOTE

        'firing_threshold': 16,       # v/
        'magnitude': 1,               # v/
        'potential_decay': .05,       # v/ Decay constant Tau=20ms, lambda=e^(-t/T)
        'prob_rand_fire': .05,        # Seemingly 0 in paper but this is critical to learning.
        'refractory_period': 0,       # v/ Gutig, Aharonov, Rotter, & Sompolinsky 2003
        'resting_mv': 0.,             # v/
        'spike_delay': 0,             # v/
        'output_range': [0, 1],       # v/

        'learning_rate': .0 / 25,   # v/ gamma_0 = gamma / Tau_z
        'max_weight': 5,              # v/
        'stdp_window': 20,            # v/ Tau_+ = Tau_- = 20ms
        'trace_decay': .04,           # v/ T_z = 25, lambda = e^(-1/T_z)

        'action_threshold': 0,        # v/ Irrelevant
    }
    _template_parts = {
        'inputs': input.RateMap,# Poisson
        'neurons': neuron.Neuron,       # v/
        'synapses': synapse.RLSTDPET,          # v/
        'weights': weight.Manual,             # v/
        'readout': readout.Threshold,         # v/
        'rewarder': reward.MatchExpected,
        'modifiers': None,             # TODO Learning rate small while accuracy high
    }

class game_templatee(Logic):
    config = Logic.PRESETS['XOR']

    config['n_inputs'] = N_INPUTS     # v/

    config['rate_mapping'] = [0, .08] # v/ 40hz = 40spikes/500ms

##
N_RUNS = 1
successes = 0
for _ in range(N_RUNS):
    network, game = run(network_templatee, game_templatee)

In [ ]:
## Rate - Initial
experiment_params = {
    'n_episodes': 1,
    'len_episode': 50,
    'eval_steps': 50, 
}

N_INPUTS = 60
N_NEURONS = 61
N_OUTPUTS = 1


w = np.vstack((  # Fully connected, generated randomly over interval
    np.hstack((
        np.random.uniform(0, .2, (N_INPUTS, N_NEURONS - N_OUTPUTS)),
        np.zeros((N_INPUTS, N_OUTPUTS)))),
    np.hstack((
        np.zeros((N_NEURONS - N_OUTPUTS, N_NEURONS - N_OUTPUTS)),
        np.random.uniform(0, .2, (N_NEURONS - N_OUTPUTS, N_OUTPUTS)))),
    np.zeros((N_OUTPUTS, N_NEURONS)),
))
w = np.ma.array(np.float16(w), mask=(w == 0), fill_value=0)

# Timestep = 1ms
# Trace suggestion multipliers - A_+ = 1, A_- = -1
# Paper uses poisson input @ 40hz high
# Paper uses inh weights not inh neurons
# Paper seemingly has no random fires

class network_templatee(FlorianSNN):
    config = {
        'matrix': w,                  # v/

        'n_neurons': N_NEURONS,       # v/
        'input_pct_inhibitory': .5,   # v/
        'neuron_pct_inhibitory': 0,          # v/
        'processing_time': 500,       # v/ 500ms NOTENOTENOTENOTENOTENOTENOTENOTENOTENOTENOTE

        'firing_threshold': 16,       # v/
        'magnitude': 1,               # v/
        'potential_decay': .05,       # v/ Decay constant Tau=20ms, lambda=e^(-t/T)
        'prob_rand_fire': .15,        # Seemingly 0 in paper but this is critical to learning.
        'refractory_period': 0,       # v/ Gutig, Aharonov, Rotter, & Sompolinsky 2003
        'resting_mv': 0.,             # v/
        'spike_delay': 0,             # v/
        'output_range': [0, 1],       # v/

        'learning_rate': .625 / 25,   # v/ gamma_0 = gamma / Tau_z
        'max_weight': 5,              # v/
        'stdp_window': 20,            # v/ Tau_+ = Tau_- = 20ms
        'trace_decay': .04,           # v/ T_z = 25, lambda = e^(-1/T_z)

        'action_threshold': 0,        # v/ Irrelevant
    }
    _template_parts = {
        'inputs': input.RateMap,# Poisson
        'neurons': neuron.Neuron,       # v/
        'synapses': synapse.RLSTDPET,          # v/
        'weights': weight.Manual,             # v/
        'readout': readout.Threshold,         # v/
        'rewarder': reward.MatchExpected,
        'modifiers': None,             # TODO Learning rate small while accuracy high
    }

class game_templatee(Logic):
    config = Logic.PRESETS['XOR']

    config['n_inputs'] = N_INPUTS     # v/
    config['n_outputs'] = N_OUTPUTS   # v/

    config['rate_mapping'] = [0, .08] # v/ 40hz = 40spikes/500ms

for _ in range(1):
    network, game = run(network_templatee, game_templatee)

In [ ]:
## Rate - WORKING
experiment_params = {
    'n_episodes': 1,
    'len_episode': 800,
    'eval_steps': 50, 
}

N_INPUTS = 60
N_NEURONS = 61
N_OUTPUTS = 1


w = np.vstack((  # Fully connected, generated randomly over interval
    np.hstack((
        np.random.uniform(0, .2, (N_INPUTS, N_NEURONS - N_OUTPUTS)),
        np.zeros((N_INPUTS, N_OUTPUTS)))),
    np.hstack((
        np.zeros((N_NEURONS - N_OUTPUTS, N_NEURONS - N_OUTPUTS)),
        np.random.uniform(0, .2, (N_NEURONS - N_OUTPUTS, N_OUTPUTS)))),
    np.zeros((N_OUTPUTS, N_NEURONS)),
))
w = np.ma.array(np.float16(w), mask=(w == 0), fill_value=0)

# Timestep = 1ms
# Trace suggestion multipliers - A_+ = 1, A_- = -1
# Paper uses poisson input @ 40hz high
# Paper uses inh weights not inh neurons
# Paper seemingly has no random fires

class network_templatee(FlorianSNN):
    config = {
        'matrix': w,                  # v/
        'n_neurons': N_NEURONS,       # v/
        'input_pct_inhibitory': .5,   # v/
        'neuron_pct_inhibitory': 0,          # v/
        'processing_time': 500,       # v/ 500ms NOTENOTENOTENOTENOTENOTENOTENOTENOTENOTENOTE

        'firing_threshold': 16,       # v/
        'magnitude': 1,               # v/
        'potential_decay': .05,       # v/ Decay constant Tau=20ms, lambda=e^(-t/T)
        'prob_rand_fire': .15,        # Seemingly 0 in paper but this is critical to learning.
        'refractory_period': 0,       # v/ Gutig, Aharonov, Rotter, & Sompolinsky 2003
        'resting_mv': 0.,             # v/
        'spike_delay': 0,             # v/
        'output_range': [0, 1],       # v/

        'learning_rate': .625 / 25,   # v/ gamma_0 = gamma / Tau_z
        'max_weight': 5,              # v/
        'stdp_window': 20,            # v/ Tau_+ = Tau_- = 20ms
        'trace_decay': .04,           # v/ T_z = 25, lambda = e^(-1/T_z)

        'action_threshold': 0,        # v/ Irrelevant
    }
    _template_parts = {
        'inputs': input.RateMap,# Poisson
        'neurons': neuron.Neuron,       # v/
        'synapses': synapse.RLSTDPET,          # v/
        'weights': weight.Manual,             # v/
        'readout': readout.Threshold,         # v/
        'rewarder': reward.MatchExpected,
        'modifiers': None,             # TODO Learning rate small while accuracy high
    }

class game_templatee(Logic):
    config = Logic.PRESETS['XOR']

    config['n_inputs'] = N_INPUTS     # v/
    config['n_outputs'] = N_OUTPUTS   # v/

    config['rate_mapping'] = [0, .08] # v/ 40hz = 40spikes/500ms

for _ in range(1):
    network, game = run(network_templatee, game_templatee)

In [ ]:
## Temporal - initial
experiment_params = {
    'n_episodes': 1,
    'len_episode': 50,
    'eval_steps': 50,
}

N_INPUTS = 2
N_NEURONS = 21
N_OUTPUTS = 1

PROCESSING_TIME = 500
simple_map = {  # 100hz spike trains
    False: np.int_(np.random.uniform(0, 1, (PROCESSING_TIME, N_INPUTS // 2)) <= 50 * .0001),
    True: np.int_(np.random.uniform(0, 1, (PROCESSING_TIME, N_INPUTS // 2)) <= 50 * .0001),
}

input_map = {
    (A, B): np.hstack((simple_map[A], simple_map[B]))
    for A in [False, True] for B in [False, True]
}
w = np.vstack((  # Fully connected, generated randomly over interval
    np.hstack((
        np.random.uniform(0, .4, (N_INPUTS, N_NEURONS - N_OUTPUTS)),
        np.zeros((N_INPUTS, N_OUTPUTS)))),
    np.hstack((
        np.zeros((N_NEURONS - N_OUTPUTS, N_NEURONS - N_OUTPUTS)),
        np.random.uniform(0, .4, (N_NEURONS - N_OUTPUTS, N_OUTPUTS)))),
    np.zeros((N_OUTPUTS, N_NEURONS)),
))
w = np.ma.array(np.float16(w), mask=(w == 0), fill_value=0)

##
class network_templatee(FlorianSNN):
    config = {
        'matrix': w,                  # v/
        'n_neurons': N_NEURONS,       # v/
        'input_pct_inhibitory': .5,   # v/
        'neuron_pct_inhibitory': 0,          # v/
        'processing_time': PROCESSING_TIME,# v/
        'mapping': input_map,

        'firing_threshold': 16,       # v/
        'magnitude': 1,               # v/
        'potential_decay': .05,       # v/ Decay constant Tau=20ms, lambda=e^(-t/T)
        'prob_rand_fire': .15,

        'refractory_period': 0,       # v/ Gutig, Aharonov, Rotter, & Sompolinsky 2003
        'resting_mv': 0.,             # v/
        'spike_delay': 0,             # v/
        'output_range': [0, 1],       # v/

        'learning_rate': .25 / 25,   # v/ gamma_0 = gamma / Tau_z
        'max_weight': 5,              # v/
        'stdp_window': 20,            # v/ Tau_+ = Tau_- = 20ms
        'trace_decay': .04,           # v/ T_z = 25, lambda = e^(-1/T_z)

        'action_threshold': 0,        # v/ Irrelevant
    }

    _template_parts = {
        'inputs': input.StaticMap,
        'neurons': neuron.Neuron,
        'synapses': synapse.RLSTDPET,
        'weights': weight.Manual,
        'readout': readout.Threshold,
        'rewarder': reward.MatchExpected,
        'modifiers': None,
    }

class game_templatee(Logic):
    config = Logic.PRESETS['XOR']
    config['n_inputs'] = N_INPUTS
    config['n_outputs'] = N_OUTPUTS
    config['rate_mapping'] = [False, True]


##
for _ in range(1):
    network, game = run(network_templatee, game_templatee)

In [ ]:
## Temporal w/ poisson process - WORKING
experiment_params = {
    'n_episodes': 1,
    'len_episode': 800,
    'eval_steps': 50,
}

N_INPUTS = 2
N_NEURONS = 21
N_OUTPUTS = 1

PROCESSING_TIME = 500
simple_map = {  # 100hz spike trains
    False: np.int_(np.random.uniform(0, 1, (PROCESSING_TIME, N_INPUTS // 2)) <= 50 * .0001),
    True: np.int_(np.random.uniform(0, 1, (PROCESSING_TIME, N_INPUTS // 2)) <= 50 * .0001),
}

input_map = {
    (A, B): np.hstack((simple_map[A], simple_map[B]))
    for A in [False, True] for B in [False, True]
}
w = np.vstack((  # Fully connected, generated randomly over interval
    np.hstack((
        np.random.uniform(0, .4, (N_INPUTS, N_NEURONS - N_OUTPUTS)),
        np.zeros((N_INPUTS, N_OUTPUTS)))),
    np.hstack((
        np.zeros((N_NEURONS - N_OUTPUTS, N_NEURONS - N_OUTPUTS)),
        np.random.uniform(0, .4, (N_NEURONS - N_OUTPUTS, N_OUTPUTS)))),
    np.zeros((N_OUTPUTS, N_NEURONS)),
))
w = np.ma.array(np.float16(w), mask=(w == 0), fill_value=0)

##
class network_templatee(FlorianSNN):
    config = {
        'matrix': w,                  # v/
        'n_neurons': N_NEURONS,       # v/
        'input_pct_inhibitory': .5,   # v/
        'neuron_pct_inhibitory': 0,          # v/
        'processing_time': PROCESSING_TIME,# v/
        'mapping': input_map,

        'firing_threshold': 16,       # v/
        'magnitude': 1,               # v/
        'potential_decay': .05,       # v/ Decay constant Tau=20ms, lambda=e^(-t/T)
        'prob_rand_fire': .15,

        'refractory_period': 0,       # v/ Gutig, Aharonov, Rotter, & Sompolinsky 2003
        'resting_mv': 0.,             # v/
        'spike_delay': 0,             # v/
        'output_range': [0, 1],       # v/

        'learning_rate': .25 / 25,   # v/ gamma_0 = gamma / Tau_z
        'max_weight': 5,              # v/
        'stdp_window': 20,            # v/ Tau_+ = Tau_- = 20ms
        'trace_decay': .04,           # v/ T_z = 25, lambda = e^(-1/T_z)

        'action_threshold': 0,        # v/ Irrelevant
    }

    _template_parts = {
        'inputs': input.StaticMap,
        'neurons': neuron.Neuron,
        'synapses': synapse.RLSTDPET,
        'weights': weight.Manual,
        'readout': readout.Threshold,
        'rewarder': reward.MatchExpected,
        'modifiers': None,
    }

class game_templatee(Logic):
    config = Logic.PRESETS['XOR']
    config['n_inputs'] = N_INPUTS
    config['n_outputs'] = N_OUTPUTS
    config['rate_mapping'] = [False, True]


##
for _ in range(1):
    network, game = run(network_templatee, game_templatee)